<a href="https://colab.research.google.com/github/Svetorus/Spark/blob/main/SparkSQL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas для проверки результата

In [ ]:
# библиотеки
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
# загрузка из компа в google colab
uploaded = files.upload()

Saving movies.csv to movies.csv


In [ ]:
movies = pd.read_csv('movies.csv')
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# заменил(убрал) все запятые
movies['title'] = movies['title'].str.replace(',','')

In [ ]:
# Убираем все значния после круглой скобки т.е. цифры и скобки
movies['title'] = movies['title'].str.split('(', expand=True)[0]
# Убираем все пробелы в начале и в конце
movies['title'] = movies.title.str.rstrip()
# Разделяем на отдельные слова
movies =  movies.assign(title=movies['title'].str.split(' ')).explode('title')
# Считаем частоту слов в столбце
movies['title'].value_counts()

           10236
The         2129
of           971
the          947
and          343
           ...  
från           1
Songs          1
xXx            1
bacio          1
Unboxed        1
Name: title, Length: 9419, dtype: int64

In [ ]:
# Проверка
# movies[movies.title.str.contains('Christmas Carol', case=False)]

# Написать на spark(код SQL)  
к сожалению проверить не могу результат, написал код как бы я сделал.

In [ ]:
from pyspark.sql.functions import udf, initcap, regexp_extract
divide = udf(lambda row: row.split(' '), returnType=StringType())

def clean_column(df,column):
  df.withColumn(column,split(df[column],'\(').getItem(0))\
  .withColumn(column, F.regexp_replace(column, ',',''))\
  .withColumn(column,F.regexp_replace(column, '^[ ]',''))\
  .withColumn(column,F.regexp_replace(column, '[ ]$',''))\
  .withColumn(column, divide(column))\
  .groupBy(column)\
  .count()\
  .sort('count', ascending=False)\
  .show()

clean_column(data,'title')

# Использование pyspark

### Установка
Используем [образ в docker](https://hub.docker.com/r/bitnami/spark/):
1. Скачиваем себе файл docker-compose.yml:
```
curl -LO https://raw.githubusercontent.com/bitnami/bitnami-docker-spark/master/docker-compose.yml
```
2. Скачиваем образ Spark:
```
docker pull bitnami/spark:latest
```
3. Запускаем (в папке, где лежит скачанный docker-compose.yml):
```
docker-compose up
```

Для завершения работы:
```
docker-compose down
```

### Действия на master:
0. Копирование данных на master
```
docker cp cards.csv id_контейнера:/opt/bitnami/spark/
```
1. Переходим в докер со спарком
```
docker exec -it namedocker bash  
```
2 Запуск python-консоли
```
pyspark
```

3. Инициализация RDD (ленивая)
```python
data = sc.textFile('movies.csv')
```

4. Посмотреть 5 строк
```python
data.take(5)
```

5. Сколько строк в данных
```python
data.count()
```

6. Wordcount
```python
data.map(lambda line: (line.split(',')[1], 1)).reduceByKey(lambda a, b: a+b).takeOrdered(5, key=lambda x: -x[1])
```

# Ленивые вычисления

In [ ]:
data = [11, 22, 33, 44, 55]
data

[11, 22, 33, 44, 55]

In [ ]:
[x % 2 for x in data]  # все данные лежат в RAM

[1, 0, 1, 0, 1]

In [ ]:
even_odd = map(lambda x: x % 2, data)  # расхода по памяти нет
even_odd

In [ ]:
even_odd.__sizeof__()

32

In [ ]:
next(even_odd)

1

In [ ]:
next(even_odd)

0

In [ ]:
wrong_data = [11, 22, '33', 44, 55]

In [ ]:
[x % 2 for x in wrong_data]

TypeError: not all arguments converted during string formatting

In [ ]:
even_odd = map(lambda x: x % 2, wrong_data)

In [ ]:
next(even_odd)

1

In [ ]:
next(even_odd)

0

In [ ]:
next(even_odd)

In [ ]:
[
    ('червы', 1),
    ('пики', 1),
    ('пики', 1),
    ('червы', 1),
]

In [ ]:
.groupby('масть').sum()

In [ ]:
[
    ('червы', 1),  # 1
    ('червы', 1),  # 1+1 = 2
    ('червы', 1),  # 2+1 = 3
    ('червы', 1),  # 3+1 = 4
    ('червы', 1),
    ('червы', 1),
    ('червы', 1),
    
    ('пики', 1),
    ('пики', 1),
]